In [79]:
import pandas as pd

In [80]:
# read in aseg_final and mpr_final tsv files
aseg_raw = pd.read_csv('../data-for-lssm/raw/aseg_final_4_14_2025.csv', sep=',')
mpr_raw = pd.read_csv('../data-for-lssm/raw/mpr_final_4_10_2025.csv', sep=',')

In [81]:
# only keep relevant columns from each dataframe
aseg = aseg_raw[["Subject","Age","Acq Date"]]
mpr = mpr_raw[["Subject","Age","Acq Date"]]

In [82]:
# merge the two dataframes on all of the columns, making sure to keep all unique rows
aseg_mpr = pd.merge(aseg, mpr, on=["Subject","Age","Acq Date"], how='outer')
aseg_mpr.drop_duplicates(inplace=True)
aseg_mpr.reset_index(drop=True, inplace=True)
aseg_mpr

,Subject,Age,Acq Date
0,002_S_0295,85,11/02/2006
1,002_S_0295,85,4/18/2006
2,002_S_0295,86,5/25/2007
3,002_S_0295,87,7/23/2008
4,002_S_0295,88,5/22/2009
...,...,...,...
2873,941_S_1203,83,1/29/2007
2874,941_S_1203,84,10/13/2007
2875,941_S_1203,86,2/25/2009
2876,941_S_1203,87,3/27/2010


In [83]:
# print the number of unique subjects
print(f"Number of unique subjects: {aseg_mpr['Subject'].nunique()}")

Number of unique subjects: 605


In [84]:
# reformat Acq Date to YYYY-MM-DD
aseg_mpr['Acq Date'] = pd.to_datetime(aseg_mpr['Acq Date'], format='%m/%d/%Y').dt.strftime('%Y-%m-%d')
aseg_mpr

,Subject,Age,Acq Date
0,002_S_0295,85,2006-11-02
1,002_S_0295,85,2006-04-18
2,002_S_0295,86,2007-05-25
3,002_S_0295,87,2008-07-23
4,002_S_0295,88,2009-05-22
...,...,...,...
2873,941_S_1203,83,2007-01-29
2874,941_S_1203,84,2007-10-13
2875,941_S_1203,86,2009-02-25
2876,941_S_1203,87,2010-03-27


In [85]:
filtered_output = pd.read_csv('../data-for-lssm/raw/filtered_output.csv', sep=',')
filtered_output

,Subject,Session
0,002_S_0295,2006-04-18_08_20_30.0/I167591
1,002_S_0295,2006-11-02_08_16_44.0/I178297
2,002_S_0295,2007-05-25_07_12_36.0/I177649
3,002_S_0295,2008-07-23_14_51_41.0/I177659
4,002_S_0295,2009-05-22_07_00_57.0/I185652
...,...,...
4963,941_S_1203,2007-10-13_12_54_16.0/I98902
4964,941_S_1203,2009-02-25_07_14_57.0/I142502
4965,941_S_1203,2010-03-27_15_31_12.0/I204852
4966,941_S_1363,2007-03-12_12_46_38.0/I185316


In [86]:
# merge the filtered_output dataframe with the aseg_mpr dataframe just on Subject
aseg_mpr_filtered = pd.merge(filtered_output, aseg_mpr, on='Subject', how='left')
aseg_mpr_filtered


,Subject,Session,Age,Acq Date
0,002_S_0295,2006-04-18_08_20_30.0/I167591,85,2006-11-02
1,002_S_0295,2006-04-18_08_20_30.0/I167591,85,2006-04-18
2,002_S_0295,2006-04-18_08_20_30.0/I167591,86,2007-05-25
3,002_S_0295,2006-04-18_08_20_30.0/I167591,87,2008-07-23
4,002_S_0295,2006-04-18_08_20_30.0/I167591,88,2009-05-22
...,...,...,...,...
26622,941_S_1203,2010-03-27_15_31_12.0/I204852,84,2007-10-13
26623,941_S_1203,2010-03-27_15_31_12.0/I204852,86,2009-02-25
26624,941_S_1203,2010-03-27_15_31_12.0/I204852,87,2010-03-27
26625,941_S_1363,2007-03-12_12_46_38.0/I185316,70,2007-03-12


In [87]:
aseg_mpr_filtered.dtypes

Subject     object
Session     object
Age          int64
Acq Date    object
dtype: object

In [88]:
# only keep rows in aseg_mpr_filtered if the part of the Session string before the first underscore matches the value in the Acq Date column
aseg_mpr_filtered['Session_split'] = aseg_mpr_filtered['Session'].str.split('_').str[0]
aseg_mpr_filtered = aseg_mpr_filtered[aseg_mpr_filtered['Session_split'] == aseg_mpr_filtered['Acq Date']]
aseg_mpr_filtered.reset_index(drop=True, inplace=True)
aseg_mpr_filtered

,Subject,Session,Age,Acq Date,Session_split
0,002_S_0295,2006-04-18_08_20_30.0/I167591,85,2006-04-18,2006-04-18
1,002_S_0295,2006-11-02_08_16_44.0/I178297,85,2006-11-02,2006-11-02
2,002_S_0295,2007-05-25_07_12_36.0/I177649,86,2007-05-25,2007-05-25
3,002_S_0295,2008-07-23_14_51_41.0/I177659,87,2008-07-23,2008-07-23
4,002_S_0295,2009-05-22_07_00_57.0/I185652,88,2009-05-22,2009-05-22
...,...,...,...,...,...
4963,941_S_1203,2007-10-13_12_54_16.0/I98902,84,2007-10-13,2007-10-13
4964,941_S_1203,2009-02-25_07_14_57.0/I142502,86,2009-02-25,2009-02-25
4965,941_S_1203,2010-03-27_15_31_12.0/I204852,87,2010-03-27,2010-03-27
4966,941_S_1363,2007-03-12_12_46_38.0/I185316,70,2007-03-12,2007-03-12


In [89]:
# drop Acq Date and Session_split columns from aseg_mpr_filtered
aseg_mpr_filtered.drop(columns=['Acq Date', 'Session_split'], inplace=True)
aseg_mpr_filtered

/tmp/ipykernel_1962138/1275374813.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aseg_mpr_filtered.drop(columns=['Acq Date', 'Session_split'], inplace=True)


,Subject,Session,Age
0,002_S_0295,2006-04-18_08_20_30.0/I167591,85
1,002_S_0295,2006-11-02_08_16_44.0/I178297,85
2,002_S_0295,2007-05-25_07_12_36.0/I177649,86
3,002_S_0295,2008-07-23_14_51_41.0/I177659,87
4,002_S_0295,2009-05-22_07_00_57.0/I185652,88
...,...,...,...
4963,941_S_1203,2007-10-13_12_54_16.0/I98902,84
4964,941_S_1203,2009-02-25_07_14_57.0/I142502,86
4965,941_S_1203,2010-03-27_15_31_12.0/I204852,87
4966,941_S_1363,2007-03-12_12_46_38.0/I185316,70


In [90]:
# rename Subject, Session, and Age columns to participant_id, session_id, and age
aseg_mpr_filtered.rename(columns={'Subject': 'participant_id', 'Session': 'session_id', 'Age': 'age'}, inplace=True)
aseg_mpr_filtered

/tmp/ipykernel_1962138/4080103893.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aseg_mpr_filtered.rename(columns={'Subject': 'participant_id', 'Session': 'session_id', 'Age': 'age'}, inplace=True)


,participant_id,session_id,age
0,002_S_0295,2006-04-18_08_20_30.0/I167591,85
1,002_S_0295,2006-11-02_08_16_44.0/I178297,85
2,002_S_0295,2007-05-25_07_12_36.0/I177649,86
3,002_S_0295,2008-07-23_14_51_41.0/I177659,87
4,002_S_0295,2009-05-22_07_00_57.0/I185652,88
...,...,...,...
4963,941_S_1203,2007-10-13_12_54_16.0/I98902,84
4964,941_S_1203,2009-02-25_07_14_57.0/I142502,86
4965,941_S_1203,2010-03-27_15_31_12.0/I204852,87
4966,941_S_1363,2007-03-12_12_46_38.0/I185316,70


In [91]:
# save the aseg_mpr_filtered dataframe to a tsv file called adni_participants.tsv
aseg_mpr_filtered.to_csv('../data-for-lssm/cleaned/adni_participants.tsv', sep='\t', index=False)